In [103]:
#! bash install.sh
%run prelude.py

In [104]:
#
# Copied from 1.4
#

def PE_range(p, X):
    # Assumes X to be sorted
    q = p/100.0
    n = len(X)
    restrict = lambda x : max(1, min(n, x))
    a = restrict(ceil(q*n))
    b = restrict(floor(q*n) + 1)
    return (X[a-1], X[b-1])

def PE(p,X):
    # Assumes X to be sorted
    x,y = PE_range(p,X)
    return (x+y)/2

FILENAME="../datasets/api_latencies_24x1h.tsv"
XAL = defaultdict(list)
with open(FILENAME) as fh:
    for line in fh:
        # line = "3 232.123321" -- hour latency
        a, b = line.split("\t")
        XAL[int(a)].append(float(b))

        
for i in range(24):
    XAL[i] = sorted(XAL[i])

# Aggregating Percentiles

In [105]:
P = [0, 50, 90, 99, 99.9, 100]

In [123]:
H("API Latencies over time")
TAB([ [h] + [ PE(p,X) for p in P ] for h, X in XAL.items() ], headers=["hour"] + P , floatfmt=".1f" )

H("Total API Latency percentiles")
XT = []
for X in XAL.values():
    for x in X:
        XT.append(x)
        
TOTP = [ PE(p,XT) for p in P ]

TAB([["total"] + TOTP], headers=[""] + P)

  hour    0    50     90      99     99.9      100
------  ---  ----  -----  ------  -------  -------
     0  0.3  12.0   19.8   113.1    450.5   1406.1
     1  0.4  16.6   23.4   115.0    606.6   1548.5
     2  0.4  20.6   27.5   117.1    936.2   3001.0
     3  0.3  20.8   35.1   425.8   1655.4   2776.4
     4  0.4  23.2   42.7   497.2   1234.8   2195.7
     5  0.4  24.3   71.6  2877.5   3298.5   3298.5
     6  0.4  25.5   95.5   154.6    191.1    191.1
     7  7.2  26.9   87.3   125.5    125.5    125.5
     8  0.4  24.9   71.1   137.3    137.3    137.3
     9  7.7  33.7  170.2  5091.4   5091.4   5091.4
    10  7.6  25.4   73.7    83.2     83.2     83.2
    11  0.6  26.3   78.1    95.9     95.9     95.9
    12  7.5  25.2   75.8   115.3    249.3    249.3
    13  5.1  25.2   76.1   116.1    117.1    117.1
    14  0.5  25.9  105.1   277.6    277.6    277.6
    15  0.6  27.3  104.7   147.3    147.3    147.3
    16  3.0   8.1   33.4   172.6    225.3    225.3
    17  0.4  10.9   17.0   140.

             0       50       90       99     99.9      100
-----  -------  -------  -------  -------  -------  -------
total  0.29394  15.9027  25.4473  71.8173  5797.75  10990.4


In [124]:
# Averaging Percentiles

In [125]:
H("Averaging Percentiles")

AVGP = [ np.mean([  PE(p,X) for X in XAL.values() ]) for p in P  ]
TAB([ ["avg-percentiles"] + WAVGP ], headers=[""] + P)

                        0       50       90       99     99.9     100
---------------  --------  -------  -------  -------  -------  ------
avg-percentiles  0.341828  20.8396  39.6724  1307.49  3199.12  5661.6


In [126]:
H("Comparison")
HEAD="p      p-total  p-avg    err                   err-%".split()
TAB([[ P[i], TOTP[i], AVGP[i], abs(TOTP[i]-AVGP[i]), abs(TOTP[i]-AVGP[i])/TOTP[i] * 100 ] for i in range(len(P)) ], floatfmt=".1f", headers=HEAD)

    p    p-total    p-avg     err    err-%
-----  ---------  -------  ------  -------
  0.0        0.3      1.9     1.6    540.0
 50.0       15.9     22.3     6.4     40.1
 90.0       25.4     61.3    35.9    141.1
 99.0       71.8    873.5   801.6   1116.2
 99.9     5797.8   1655.3  4142.4     71.4
100.0    10990.4   2664.3  8326.1     75.8


In [127]:
H("Weighted Average Percentile")
WAVGP = [ np.sum([ len(X) * PE(p,X) for X in XAL.values() ]) / np.sum( [ len(X) for X in XAL.values() ]) for p in P  ]
TAB([ ["w-avg-percentiles"] + WAVGP ], headers=[""] + P)

                          0       50       90       99     99.9     100
-----------------  --------  -------  -------  -------  -------  ------
w-avg-percentiles  0.341828  20.8396  39.6724  1307.49  3199.12  5661.6


In [128]:
H("Comparison")
HEAD="p      p-total  p-wavg   err                   err-%".split()
TAB([[ P[i], TOTP[i], AVGP[i], abs(TOTP[i]-WAVGP[i]), abs(TOTP[i]-WAVGP[i])/TOTP[i] * 100 ] for i in range(len(P)) ], floatfmt=".1f", headers=HEAD)

    p    p-total    p-wavg     err    err-%
-----  ---------  --------  ------  -------
  0.0        0.3       1.9     0.0     16.3
 50.0       15.9      22.3     4.9     31.0
 90.0       25.4      61.3    14.2     55.9
 99.0       71.8     873.5  1235.7   1720.6
 99.9     5797.8    1655.3  2598.6     44.8
100.0    10990.4    2664.3  5328.8     48.5


# Discussion

* Averaging Percentiles does not work at all.

* Contrast this to the situation with the count-below metrics, which carry similar information

* Same holds for: median, IQR, etc.

Yet, calculating accurate percentiles over aggergated datasets is necessary for basic analysis of API performance:

- Aggregation across time
- Aggregation across different nodes
- Aggregation across different endpoints

# Percentiles can't be Aggregated

* Blog: www.circonus.com/problem-math/

* Twitter: https://twitter.com/heinrichhartman/status/748562001392111617

* Reddit: https://www.reddit.com/r/devops/comments/941n2k/tsdbs_at_scale_part_one/e3po8d3/

* Gil Tene - How not to measure latency -- https://www.youtube.com/watch?v=lJ8ydIuPFeU&feature=youtu.be&t=9m15s".

# Mergable Percentile Summaries

There are multiple methods available to summarize data in a mergable way, that allows for percentile calculations.

1. Circllhists -- Circonus 2011

2. HDR Histograms -- Gil Tene @ Azul Systems, 2015

3. t-digest -- Dunning @ MapR, ? @ Wavefront

...

4. DD-sketch -- ? @ DataDog 2019
